<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
    Machine learning <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2024<br>
<font color=3C99D size=5>
    Practical Assignment 5 - NLP - Transformer & Bert <br>
</div>
<div dir=ltr align=center>
<font color=0CBCDF size=4>
   &#x1F349; Masoud Tahmasbi  &#x1F349;  &#x1F353; Arash Ziyaei &#x1F353;
<br>
<font color=0CBCDF size=4>
   &#x1F335; Amirhossein Akbari  &#x1F335;
</div>

____

<font color=9999FF size=4>
&#x1F388; Full Name : Farzan Rahmani
<br>
<font color=9999FF size=4>
&#x1F388; Student Number : 403210725

<font color=0080FF size=3>
This notebook covers two key topics. First, we implement a transformer model from scratch and apply it to a specific task. Second, we fine-tune the BERT model using LoRA for efficient adaptation to a downstream task.
</font>
<br>

**Note:**
<br>
<font color=66B2FF size=2>In this notebook, you are free to use any function or model from PyTorch to assist with the implementation. However, TensorFlow is not permitted for this exercise. This ensures consistency and alignment with the tools being focused on.</font>
<br>
<font color=red size=3>**Run All Cells Before Submission**</font>: <font color=FF99CC size=2>Before saving and submitting your notebook, please ensure you run all cells from start to finish. This practice guarantees that your notebook is self-consistent and can be evaluated correctly by others.</font>

# Section 1: Transformer

The transformer architecture consists of two main components: an encoder and a decoder. Each of these components is made up of multiple layers that include self-attention mechanisms and feedforward neural networks. The self-attention mechanism is central to the transformer, as it enables the model to assess the importance of different words in a sentence by considering their relationships with one another.


In this assignment, you should design a transformer model from scratch. You are required to implement the Encoder and Decoder components of a Transformer model.

In [3]:
!pip install datasets -q

In [4]:
# Importing libraries

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

# Math
import math

# HuggingFace libraries
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

# Pathlib
from pathlib import Path

# typing
from typing import Any

# Library for progress bars in loops
from tqdm import tqdm

# Importing library of warnings
import warnings

## Part 1: Input Embeddings
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">When we observe the Transformer architecture image above, we can see that the Embeddings represent the first step of both blocks.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>InputEmbedding</code> class below is responsible for converting the input text into numerical vectors of <code>d_model</code> dimensions. To prevent that our input embeddings become extremely small, we normalize them by multiplying them by the $\sqrt{d_{model}}$.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the image below, we can see how the embeddings are created. First, we have a sentence that gets split into tokens—we will explore what tokens are later on—. Then, the token IDs—identification numbers—are transformed into the embeddings, which are high-dimensional vectors.</p>

In [5]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a class `InputEmbeddings` inheriting from `nn.Module`
# - Initialize the class with two parameters:
#   1. `d_model`: Dimension of the embedding vectors
#   2. `vocab_size`: Size of the vocabulary
# - Create an embedding layer using `nn.Embedding` to map input indices to dense vectors

# - In the `forward` method:
#   1. Pass the input `x` through the embedding layer
#   2. Scale the embeddings by the square root of `d_model` for variance normalization

######################  TODO  ########################
######################  TODO  ########################
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch_size, seq_len) --> (batch_size, seq_len, d_model)
        return self.embedding(x) * math.sqrt(self.d_model)

## Part 2: positional encoding
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the original paper, the authors add the positional encodings to the input embeddings at the bottom of both the encoder and decoder blocks so the model can have some information about the relative or absolute position of the tokens in the sequence. The positional encodings have the same dimension $d_{model}$ as the embeddings, so that the two vectors can be summed and we can combine the semantic content from the word embeddings and positional information from the positional encodings.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>PositionalEncoding</code> class below, we will create a matrix of positional encodings <code>pe</code> with dimensions <code>(seq_len, d_model)</code>. We will start by filling it with $0$s.We will then apply the sine function to even indices of the positional encoding matrix while the cosine function is applied to the odd ones.</p>

<p style="
    margin-bottom: 5;
    font-size: 22px;
    font-weight: 300;
    font-family: 'Helvetica Neue', sans-serif;
    color: #000000;
  ">
    \begin{equation}
    \text{Odd Indices } (2i + 1): \quad \text{PE(pos, } 2i + 1) = \cos\left(\frac{\text{pos}}{10000^{2i / d_{model}}}\right)
    \end{equation}
</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We apply the sine and cosine functions because it allows the model to determine the position of a word based on the position of other words in the sequence, since for any fixed offset $k$, $PE_{pos + k}$ can be represented as a linear function of $PE_{pos}$. This happens due to the properties of sine and cosine functions, where a shift in the input results in a predictable change in the output.</p>

In [6]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `PositionalEncoding` class inheriting from `nn.Module`
# - Initialize with `d_model`, `seq_len`, and `dropout`
# - Generate a positional encoding matrix using sine and cosine functions
# - Register the positional encoding as a non-trainable buffer
# - In `forward`, add positional encoding to input and apply dropout

######################  TODO  ########################
######################  TODO  ########################
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len, 1)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term)
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0)  # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

## Part 3: layer normalization
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">When we look at the encoder and decoder blocks, we see several normalization layers called <b><i>Add &amp; Norm</i></b>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>LayerNormalization</code> class below performs layer normalization on the input data. During its forward pass, we compute the mean and standard deviation of the input data. We then normalize the input data by subtracting the mean and dividing by the standard deviation plus a small number called epsilon to avoid any divisions by zero. This process results in a normalized output with a mean 0 and a standard deviation 1.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will then scale the normalized output by a learnable parameter <code>alpha</code> and add a learnable parameter called <code>bias</code>. The training process is responsible for adjusting these parameters. The final result is a layer-normalized tensor, which ensures that the scale of the inputs to layers in the network is consistent.</p>

In [7]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `LayerNormalization` class inheriting from `nn.Module`
# - Initialize with `eps` (small value to prevent division by zero)
# - Define trainable parameters:
#   1. `alpha`: Scaling factor initialized to 1
#   2. `bias`: Offset initialized to 0

# - In `forward`, perform layer normalization:
#   1. Compute mean and standard deviation along the last dimension
#   2. Normalize the input using the computed mean and std
#   3. Scale and shift using `alpha` and `bias`

######################  TODO  ########################
######################  TODO  ########################
class LayerNormalization(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1))  # Multiplicative factor
        self.bias = nn.Parameter(torch.zeros(1))  # Additive factor

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

## Part 4: Feed Forward Network
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the fully connected feed-forward network, we apply two linear transformations with a ReLU activation in between. We can mathematically represent this operation as:</p>

<p style="
    margin-bottom: 5;
    font-size: 22px;
    font-weight: 300;
    font-family: 'Helvetica Neue', sans-serif;
    color: #000000;
  ">
    \begin{equation}
    \text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
    \end{equation}
</p>


<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">$W_1$ and $W_2$ are the weights, while $b_1$ and $b_2$ are the biases of the two linear transformations.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>FeedForwardBlock</code> below, we will define the two linear transformations—<code>self.linear_1</code> and <code>self.linear_2</code>—and the inner-layer <code>d_ff</code>. The input data will first pass through the <code>self.linear_1</code> transformation, which increases its dimensionality from <code>d_model</code> to <code>d_ff</code>. The output of this operation passes through the ReLU activation function, which introduces non-linearity so the network can learn more complex patterns, and the <code>self.dropout</code> layer is applied to mitigate overfitting. The final operation is the <code>self.linear_2</code> transformation to the dropout-modified tensor, which transforms it back to the original <code>d_model</code> dimension.</p>

In [8]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `FeedForwardBlock` class inheriting from `nn.Module`
# - Initialize with `d_model`, `d_ff`, and `dropout`
# - Define:
#   1. `linear_1`: Linear layer projecting from `d_model` to `d_ff`
#   2. Dropout layer for regularization
#   3. `linear_2`: Linear layer projecting back from `d_ff` to `d_model`

# - In `forward`, apply the following steps:
#   1. Pass input through `linear_1` followed by ReLU activation
#   2. Apply dropout
#   3. Pass through `linear_2` to return to original dimensions

######################  TODO  ########################
######################  TODO  ########################
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)  # W1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)  # W2 and b2

    def forward(self, x):
        # (batch_size, seq_len, d_model) --> (batch_size, seq_len, d_ff) --> (batch_size, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

## Part 5: Multi Head Attention
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The Multi-Head Attention is the most crucial component of the Transformer. It is responsible for helping the model to understand complex relationships and patterns in the data.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The image below displays how the Multi-Head Attention works. It doesn't include <code>batch</code> dimension because it only illustrates the process for one single sentence.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The Multi-Head Attention block receives the input data split into queries, keys, and values organized into matrices $Q$, $K$, and $V$. Each matrix contains different facets of the input, and they have the same dimensions as the input.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We then linearly transform each matrix by their respective weight matrices $W^Q$, $W^K$, and $W^V$. These transformations will result in new matrices $Q'$, $K'$, and $V'$, which will be split into smaller matrices corresponding to different heads $h$, allowing the model to attend to information from different representation subspaces in parallel. This split creates multiple sets of queries, keys, and values for each head.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Finally, we concatenate every head into an $H$ matrix, which is then transformed by another weight matrix $W^o$ to produce the multi-head attention output, a matrix $MH-A$ that retains the input dimensionality.</p>

In [10]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `MultiHeadAttentionBlock` class inheriting from `nn.Module`
# - Initialize with `d_model` (model dimensions), `h` (number of heads), and `dropout`:
#   1. Assert `d_model` is divisible by `h`
#   2. Define `d_k` as dimensions per head
#   3. Create weight matrices (`w_q`, `w_k`, `w_v`, `w_o`) for query, key, value, and output
#   4. Add a dropout layer for regularization

# - Implement a static `attention` method to:
#   1. Compute scaled dot-product attention
#   2. Apply mask if provided
#   3. Apply softmax and dropout
#   4. Return weighted values and attention scores

# - In `forward`, perform:
#   1. Linear transformation of input into query, key, and value
#   2. Split into `h` heads and rearrange dimensions
#   3. Compute attention output and scores using `attention`
#   4. Combine heads and apply output weight matrix

######################  TODO  ########################
######################  TODO  ########################
class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.h = h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h
        self.w_q = nn.Linear(d_model, d_model)  # Wq
        self.w_k = nn.Linear(d_model, d_model)  # Wk
        self.w_v = nn.Linear(d_model, d_model)  # Wv
        self.w_o = nn.Linear(d_model, d_model)  # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # (batch_size, h, seq_len, d_k) --> (batch_size, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1)  # (batch_size, h, seq_len, seq_len)
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q)  # (batch_size, seq_len, d_model) --> (batch_size, seq_len, d_model)
        key = self.w_k(k)  # (batch_size, seq_len, d_model) --> (batch_size, seq_len, d_model)
        value = self.w_v(v)  # (batch_size, seq_len, d_model) --> (batch_size, seq_len, d_model)

        # (batch_size, seq_len, d_model) --> (batch_size, seq_len, h, d_k) --> (batch_size, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # (batch_size, h, seq_len, d_k) --> (batch_size, seq_len, h, d_k) --> (batch_size, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # (batch_size, seq_len, d_model) --> (batch_size, seq_len, d_model)
        return self.w_o(x)

## Part 6: Residual Connection
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">When we look at the architecture of the Transformer, we see that each sub-layer, including the <i>self-attention</i> and <i>Feed Forward</i> blocks, adds its output to its input before passing it to the <i>Add &amp; Norm</i> layer. This approach integrates the output with the original input in the <i>Add &amp; Norm</i> layer. This process is known as the skip connection, which allows the Transformer to train deep networks more effectively by providing a shortcut for the gradient to flow through during backpropagation.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>ResidualConnection</code> class below is responsible for this process.</p>

In [11]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `ResidualConnection` class inheriting from `nn.Module`
# - Initialize with `dropout`:
#   1. Add a dropout layer for regularization
#   2. Include a layer normalization instance

# - In `forward`:
#   1. Normalize the input using the normalization layer
#   2. Pass the normalized input through the sublayer
#   3. Apply dropout and add the result back to the original input for residual connection

######################  TODO  ########################
######################  TODO  ########################
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

## Part 7: Encoder
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will now build the encoder. We create the <code>EncoderBlock</code> class, consisting of the Multi-Head Attention and Feed Forward layers, plus the residual connections.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the original paper, the Encoder Block repeats six times. We create the <code>Encoder</code> class as an assembly of multiple <code>EncoderBlock</code>s. We also add layer normalization as a final step after processing the input through all its blocks.</p>

In [12]:
######################  TODO  ########################
######################  TODO  ########################

# - Create an `EncoderBlock` class inheriting from `nn.Module`
# - Initialize with:
#   1. `self_attention_block`: Multi-head attention block
#   2. `feed_forward_block`: Feed-forward block
#   3. `dropout`: Dropout rate for residual connections
# - Define two residual connections for:
#   1. Self-attention block
#   2. Feed-forward block

# - In `forward`:
#   1. Apply the first residual connection with the self-attention block
#   2. Apply the second residual connection with the feed-forward block
#   3. Return the updated tensor after both layers

######################  TODO  ########################
######################  TODO  ########################
class EncoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

In [13]:
######################  TODO  ########################
######################  TODO  ########################

# - Create an `Encoder` class inheriting from `nn.Module`
# - Initialize with:
#   1. `layers`: A list of `EncoderBlock` instances
#   2. A layer normalization instance for output normalization

# - In `forward`:
#   1. Pass the input tensor `x` through each `EncoderBlock` in `self.layers`
#   2. Apply the mask during each block's forward pass
#   3. Normalize the final output and return it

######################  TODO  ########################
######################  TODO  ########################
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

## Part 8: Decoder
<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Similarly, the Decoder also consists of several DecoderBlocks that repeat six times in the original paper. The main difference is that it has an additional sub-layer that performs multi-head attention with a <i>cross-attention</i> component that uses the output of the Encoder as its keys and values while using the Decoder's input as queries.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">For the Output Embedding, we can use the same <code>InputEmbeddings</code> class we use for the Encoder. You can also notice that the self-attention sub-layer is <i>masked</i>, which restricts the model from accessing future elements in the sequence.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will start by building the <code>DecoderBlock</code> class, and then we will build the <code>Decoder</code> class, which will assemble multiple <code>DecoderBlock</code>s.</p>

In [14]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `DecoderBlock` class inheriting from `nn.Module`
# - Initialize with:
#   1. `self_attention_block`: Multi-head self-attention block
#   2. `cross_attention_block`: Multi-head cross-attention block
#   3. `feed_forward_block`: Feed-forward block
#   4. `dropout`: Dropout rate
# - Define three residual connections for:
#   1. Self-attention block
#   2. Cross-attention block
#   3. Feed-forward block

# - In `forward`:
#   1. Apply the self-attention block with target mask and residual connection
#   2. Apply the cross-attention block with source mask and residual connection
#   3. Apply the feed-forward block with residual connection
#   4. Return the updated tensor

######################  TODO  ########################
######################  TODO  ########################
class DecoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [15]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `Decoder` class inheriting from `nn.Module`
# - Initialize with:
#   1. `layers`: A list of `DecoderBlock` instances
#   2. A layer normalization instance for the final output

# - In `forward`:
#   1. Pass the input tensor `x` through each `DecoderBlock` in `self.layers`
#   2. Provide `encoder_output`, `src_mask`, and `tgt_mask` to each block
#   3. Normalize the final output using the layer normalization
#   4. Return the normalized output

######################  TODO  ########################
######################  TODO  ########################
class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">You can see in the Decoder image that after running a stack of <code>DecoderBlock</code>s, we have a Linear Layer and a Softmax function to the output of probabilities. The <code>ProjectionLayer</code> class below is responsible for converting the output of the model into a probability distribution over the <i>vocabulary</i>, where we select each output token from a vocabulary of possible tokens.</p>

In [16]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `ProjectionLayer` class inheriting from `nn.Module`
# - Initialize with:
#   1. `d_model`: Dimension of the model
#   2. `vocab_size`: Size of the output vocabulary
# - Define a linear layer to project from `d_model` to `vocab_size`

# - In `forward`:
#   1. Pass the input through the linear layer
#   2. Apply log Softmax along the last dimension
#   3. Return the log probabilities

######################  TODO  ########################
######################  TODO  ########################
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # (batch_size, seq_len, d_model) --> (batch_size, seq_len, vocab_size)
        return torch.log_softmax(self.proj(x), dim=-1)

## Part 9: Building the Transformer

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We finally have every component of the Transformer architecture ready. We may now construct the Transformer by putting it all together.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>Transformer</code> class below, we will bring together all the components of the model's architecture.</p>

In [17]:
######################  TODO  ########################
######################  TODO  ########################

# - Create a `Transformer` class inheriting from `nn.Module`
# - Initialize with:
#   1. `encoder`: Encoder module
#   2. `decoder`: Decoder module
#   3. `src_embed` and `tgt_embed`: Input embeddings for source and target languages
#   4. `src_pos` and `tgt_pos`: Positional encodings for source and target languages
#   5. `projection_layer`: Linear projection layer for final output

# - Define the `encode` method:
#   1. Apply source embeddings to input
#   2. Add positional encoding
#   3. Pass through the encoder with the source mask
#   4. Return the encoded representation

# - Define the `decode` method:
#   1. Apply target embeddings to input
#   2. Add positional encoding
#   3. Pass through the decoder with encoder output, source mask, and target mask
#   4. Return the decoder's output

# - Define the `project` method:
#   1. Pass decoder output through the projection layer
#   2. Apply log Softmax to obtain probabilities

######################  TODO  ########################
######################  TODO  ########################
class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        return self.projection_layer(x)

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The architecture is finally ready. We now define a function called <code>build_transformer</code>, in which we define the parameters and everything we need to have a fully operational Transformer model for the task of <b>machine translation</b>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will set the same parameters as in the original paper, <a href = "https://arxiv.org/pdf/1706.03762.pdf"><i>Attention Is All You Need</i></a>, where $d_{model}$ = 512, $N$ = 6, $h$ = 8, dropout rate $P_{drop}$ = 0.1, and $d_{ff}$ = 2048.</p>

In [18]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `build_transformer` function with parameters for:
#   1. Vocabulary sizes (`src_vocab_size`, `tgt_vocab_size`)
#   2. Sequence lengths (`src_seq_len`, `tgt_seq_len`)
#   3. Model dimensions (`d_model`, `d_ff`)
#   4. Number of layers (`N`) and heads (`h`)
#   5. Dropout rate (`dropout`)

# - Create:
#   1. Source and target embedding layers
#   2. Positional encoding layers for source and target
#   3. Encoder blocks with self-attention and feed-forward layers
#   4. Decoder blocks with self-attention, cross-attention, and feed-forward layers
#   5. Encoder and Decoder modules using the blocks
#   6. Projection layer to map decoder output to target vocabulary

# - Assemble all components into a `Transformer` instance
# - Initialize parameters with Xavier uniform initialization
# - Return the initialized Transformer

######################  TODO  ########################
######################  TODO  ########################

def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

The model is now ready to be trained!

## Part 10: Tokenizer

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Tokenization is a crucial preprocessing step for our Transformer model. In this step, we convert raw text into a number format that the model can process.  </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">There are several Tokenization strategies. We will use the <i>word-level tokenization</i> to transform each word in a sentence into a token.</p>

<center>
    <img src = "https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F8d5e749c-b0bd-4496-85a1-9b4397ad935f_1400x787.jpeg" width = 800, height= 800>
<p style = "font-size: 16px;
            font-family: 'Georgia', serif;
            text-align: center;
            margin-top: 10px;">Different tokenization strategies. Source: <a href = "https://shaankhosla.substack.com/p/talking-tokenization">shaankhosla.substack.com</a>.</p>
</center>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">After tokenizing a sentence, we map each token to an unique integer ID based on the created vocabulary present in the training corpus during the training of the tokenizer. Each integer number represents a specific word in the vocabulary.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Besides the words in the training corpus, Transformers use special tokens for specific purposes. These are some that we will define right away:</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>• [UNK]:</b> This token is used to identify an unknown word in the sequence.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>• [PAD]:</b> Padding token to ensure that all sequences in a batch have the same length, so we pad shorter sentences with this token. We use attention masks to <i>"tell"</i> the model to ignore the padded tokens during training since they don't have any real meaning to the task.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>•  [SOS]:</b> This is a token used to signal the <i>Start of Sentence</i>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px"><b>•  [EOS]:</b> This is a token used to signal the <i>End of Sentence</i>.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>build_tokenizer</code> function below, we ensure a tokenizer is ready to train the model. It checks if there is an existing tokenizer, and if that is not the case, it trains a new tokenizer.</p>

In [19]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `build_tokenizer` function with parameters for:
#   1. `config`: Configuration containing tokenizer file path
#   2. `ds`: Dataset to train the tokenizer
#   3. `lang`: Language for which the tokenizer is built

# - Check if the tokenizer file exists:
#   1. If not, create a new tokenizer:
#      - Initialize a word-level tokenizer with an unknown token (`[UNK]`)
#      - Set the pre-tokenizer to split text by whitespace
#      - Define a trainer with special tokens and minimum frequency
#      - Train the tokenizer on all sentences in the dataset
#      - Save the trained tokenizer to the specified file path
#   2. If the file exists, load the tokenizer from the file

# - Return the loaded or trained tokenizer

######################  TODO  ########################
######################  TODO  ########################
def build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not tokenizer_path.exists():
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

## Part 11: Load Dataset

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">For this task, we will use the <a href = "opus_books · Datasets at Hugging Face">OpusBooks dataset</a>, available on 🤗Hugging Face. This dataset consists of two features, <code>id</code> and <code>translation</code>. The <code>translation</code> feature contains pairs of sentences in different languages, such as Spanish and Portuguese, English and French, and so forth.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">I first tried translating sentences from English to Portuguese—my native tongue — but there are only 1.4k examples for this pair, so the results were not satisfying in the current configurations for this model. I then tried to use the English-French pair due to its higher number of examples—127k—but it would take too long to train with the current configurations. I then opted to train the model on the English-Italian pair, the same one used in the <a href = "https://youtu.be/ISNdQcPhsts?si=253J39cose6IdsLv">Coding a Transformer from scratch on PyTorch, with full explanation, training and inference
</a> video, as that was a good balance between performance and time of training.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We start by defining the <code>get_all_sentences</code> function to iterate over the dataset and extract the sentences according to the language pair defined—we will do that later.</p>

In [20]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `get_all_sentences` function to extract sentences from a dataset
# - Accept parameters:
#   1. `ds`: The dataset containing translation pairs
#   2. `lang`: The language key to extract translations

# - Iterate through the dataset:
#   1. Access the 'translation' field of each pair
#   2. Yield the sentence corresponding to the specified language key

######################  TODO  ########################
######################  TODO  ########################
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>get_ds</code> function is defined to load and prepare the dataset for training and validation. In this function, we build or load the tokenizer, split the dataset, and create DataLoaders, so the model can successfully iterate over the dataset in batches. The result of these functions is tokenizers for the source and target languages plus the DataLoader objects.</p>

In [21]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `get_ds` function to process and prepare the dataset for training
# - Load the `OpusBooks` dataset using:
#   1. Source and target languages from `config`
#   2. Train split of the dataset

# - Build or load tokenizers for source and target languages using `build_tokenizer`

# - Split the dataset into training and validation sets:
#   1. Allocate 90% for training and 10% for validation
#   2. Use `random_split` for randomized splitting

# - Process the splits using a `BilingualDataset` class:
#   1. Convert sentences to tokenized representations
#   2. Apply source and target tokenizers
#   3. Ensure sequence lengths conform to `config`

# - Compute and print the maximum sentence lengths for both source and target languages

# - Create DataLoader objects for training and validation:
#   1. Define batch sizes from `config`
#   2. Enable shuffling for training DataLoader

# - Return:
#   1. Training DataLoader
#   2. Validation DataLoader
#   3. Tokenizer for source language
#   4. Tokenizer for target language

######################  TODO  ########################
######################  TODO  ########################

def get_ds(config):
    ds_raw = load_dataset('opus_books', f"{config['lang_src']}-{config['lang_tgt']}", split='train')

    # Build tokenizers
    tokenizer_src = build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = build_tokenizer(config, ds_raw, config['lang_tgt'])

    # Split dataset into training and validation
    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    # Find the maximum length of each sentence in the source and target sentence
    max_len_src = 0
    max_len_tgt = 0
    for item in ds_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')

    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We define the <code>casual_mask</code> function to create a mask for the attention mechanism of the decoder. This mask prevents the model from having information about future elements in the sequence. </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We start by making a square grid filled with ones. We determine the grid size with the <code>size</code> parameter. Then, we change all the numbers above the main diagonal line to zeros. Every number on one side becomes a zero, while the rest remain ones. The function then flips all these values, turning ones into zeros and zeros into ones. This process is crucial for models that predict future tokens in a sequence.</p>

In [22]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `casual_mask` function to create an upper triangular mask
# - Accept `size` as the dimension of the square matrix
# - Steps:
#   1. Create a square matrix of size `size x size` filled with ones
#   2. Use `torch.triu` to make it upper triangular, with zeros below the diagonal
#   3. Convert the matrix to integer type
#   4. Return the mask where zeros represent the causal positions

######################  TODO  ########################
######################  TODO  ########################
def causal_mask(size):
    mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
    return mask == 0

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>BilingualDataset</code> class processes the texts of the target and source languages in the dataset by tokenizing them and adding all the necessary special tokens. This class also certifies that the sentences are within a maximum sequence length for both languages and pads all necessary sentences.</p>

In [23]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `BilingualDataset` class inheriting from `Dataset`
# - Initialize with:
#   1. `ds`: Dataset containing sentence pairs
#   2. `tokenizer_src` and `tokenizer_tgt`: Tokenizers for source and target languages
#   3. `src_lang` and `tgt_lang`: Language identifiers
#   4. `seq_len`: Maximum sequence length for tokens

# - Define special tokens (`[SOS]`, `[EOS]`, `[PAD]`) using the target tokenizer

# - Implement `__len__` to return the number of sentence pairs in the dataset

# - Implement `__getitem__` to:
#   1. Retrieve source and target texts based on the index
#   2. Tokenize source and target texts
#   3. Compute required padding for source and target tokens
#   4. Raise an error if tokenized sentences exceed `seq_len`
#   5. Build `encoder_input` by concatenating `[SOS]`, tokenized text, `[EOS]`, and padding
#   6. Build `decoder_input` by concatenating `[SOS]`, tokenized text, and padding
#   7. Build `label` by concatenating tokenized text, `[EOS]`, and padding
#   8. Ensure all tensors are of length `seq_len`

# - Return a dictionary containing:
#   1. `encoder_input`: Tensor for the encoder
#   2. `decoder_input`: Tensor for the decoder
#   3. `encoder_mask`: Mask for non-padding tokens in the encoder
#   4. `decoder_mask`: Mask for non-padding tokens in the decoder with causal masking
#   5. `label`: Expected output for training
#   6. `src_text` and `tgt_text`: Original source and target texts

######################  TODO  ########################
######################  TODO  ########################
class BilingualDataset(Dataset):
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.seq_len = seq_len

        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        # Transform the text into tokens
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        # Add sos, eos and padding to each sentence
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # We will add <s> and </s>
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1  # We will add <s>

        # Make sure the number of padding tokens is not negative. If it is, the sentence is too long
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        # Add <s> and </s> token
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only <s> token
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only </s> token
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Double check the size of the tensors to make sure they are all seq_len long
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),  # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),  # (1, seq_len) & (1, seq_len, seq_len)
            "label": label,  # (seq_len)
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

## Part 12: Validation Loop

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will now create two functions for the validation loop. The validation loop is crucial to evaluate model performance in translating sentences from data it has not seen during training.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We will define two functions. The first function, <code>greedy_decode</code>, gives us the model's output by obtaining the most probable next token. The second function, <code>run_validation</code>, is responsible for running the validation process in which we decode the model's output and compare it with the reference text for the target sentence.</p>

In [24]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `greedy_decode` function to generate the most probable sequence using a trained model
# - Accept parameters:
#   1. `model`: Trained Transformer model
#   2. `source`: Source input sequence
#   3. `source_mask`: Mask for the source sequence
#   4. `tokenizer_src` and `tokenizer_tgt`: Tokenizers for source and target languages
#   5. `max_len`: Maximum sequence length for the output
#   6. `device`: Device to run the computation (e.g., CPU or GPU)

# - Steps:
#   1. Retrieve indices for `[SOS]` and `[EOS]` tokens from the target tokenizer
#   2. Compute encoder output for the source sequence
#   3. Initialize decoder input with `[SOS]`
#   4. Loop until `max_len` is reached or `[EOS]` is generated:
#      - Create a causal mask for the decoder input
#      - Compute decoder output using encoder output and masks
#      - Apply the projection layer to get probabilities for the next token
#      - Select the token with the highest probability and append it to the decoder input
#      - Break the loop if `[EOS]` is generated
#   5. Return the generated sequence of tokens

######################  TODO  ########################
######################  TODO  ########################
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # Build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # Calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # Get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)

In [25]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `run_validation` function to evaluate the model on the validation dataset
# - Accept parameters:
#   1. `model`: Trained Transformer model
#   2. `validation_ds`: Validation dataset
#   3. `tokenizer_src` and `tokenizer_tgt`: Tokenizers for source and target languages
#   4. `max_len`: Maximum sequence length for decoding
#   5. `device`: Device to run the computation (e.g., CPU or GPU)
#   6. `print_msg`: Function for displaying output messages
#   7. `global_state`: Optional global state for tracking progress
#   8. `writer`: Optional logging writer (e.g., TensorBoard)
#   9. `num_examples`: Number of examples to process per run (default: 2)

# - Steps:
#   1. Set the model to evaluation mode
#   2. Initialize a counter to track the number of processed examples
#   3. Define a fixed console width for printed messages
#   4. Iterate through the validation dataset:
#      - Retrieve `encoder_input` and `encoder_mask` and move them to the specified device
#      - Ensure batch size is 1 for validation
#      - Use `greedy_decode` to generate the model's predictions
#      - Decode the model's output into human-readable text
#      - Print source, target, and predicted text using `print_msg`
#      - Break the loop after processing the specified number of examples (`num_examples`)
#   5. Ensure no gradients are computed during evaluation

######################  TODO  ########################
######################  TODO  ########################
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device)  # (batch_size, seq_len)
            encoder_mask = batch["encoder_mask"].to(device)  # (batch_size, 1, 1, seq_len)

            # Check that the batch size is 1
            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            # Print the source, target and model output
            print_msg('-' * console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                break

## Part 13: Training Loop

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We are ready to train our Transformer model on the OpusBook dataset for the English to Italian translation task.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We first start by defining the <code>get_model</code> function to load the model by calling the <code>build_transformer</code> function we have previously defined. This function uses the <code>config</code> dictionary to set a few parameters.</p>

In [26]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `get_model` function to initialize a Transformer model
# - Accept parameters:
#   1. `config`: Configuration dictionary with model settings
#   2. `vocab_src_len`: Length of the source language vocabulary
#   3. `vocab_tgt_len`: Length of the target language vocabulary

# - Use the `build_transformer` function to:
#   1. Create a Transformer model
#   2. Pass the source and target vocabulary lengths
#   3. Set sequence length (`seq_len`) and embedding dimensionality (`d_model`) from `config`

# - Return the initialized model

######################  TODO  ########################
######################  TODO  ########################
def get_model(config, vocab_src_len, vocab_tgt_len):
    model = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'], config['seq_len'], config['d_model'])
    return model

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">I have mentioned the <code>config</code> dictionary several times throughout this notebook. Now, it is time to create it.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the following cell, we will define two functions to configure our model and the training process.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In the <code>get_config</code> function, we define crucial parameters for the training process. <code>batch_size</code> for the number of training examples used in one iteration, <code>num_epochs</code> as the number of times the entire dataset is passed forward and backward through the Transformer, <code>lr</code> as the learning rate for the optimizer, etc. We will also finally define the pairs from the OpusBook dataset, <code>'lang_src': 'en'</code> for selecting English as the source language and <code>'lang_tgt': 'it'</code> for selecting Italian as the target language.</p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">The <code>get_weights_file_path</code> function constructs the file path for saving or loading model weights for any specific epoch.</p>

In [27]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `get_config` function to return a dictionary of settings for building and training the Transformer model:
#   1. `batch_size`: Number of samples per training batch
#   2. `num_epochs`: Total training epochs
#   3. `lr`: Learning rate for optimization
#   4. `seq_len`: Maximum sequence length for tokens
#   5. `d_model`: Dimensionality of embeddings (e.g., 512)
#   6. `lang_src` and `lang_tgt`: Source and target languages
#   7. `model_folder`: Folder to save model weights
#   8. `model_basename`: Base name for model files
#   9. `preload`: Option to preload a model (default: None)
#   10. `tokenizer_file`: Filename pattern for saving tokenizers
#   11. `experiment_name`: Name of the experiment for logging

# - Define `get_weights_file_path` to construct a file path for saving/retrieving model weights:
#   1. Accept `config` dictionary and `epoch` string as parameters
#   2. Retrieve `model_folder` and `model_basename` from `config`
#   3. Construct the filename with the base name and epoch
#   4. Combine the current directory, model folder, and filename to return the full path

######################  TODO  ########################
######################  TODO  ########################
def get_config():
    return {
        "batch_size": 24,
        "num_epochs": 4,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": None,
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">We finally define our last function, <code>train_model</code>, which takes the <code>config</code> arguments as input. </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">In this function, we will set everything up for the training. We will load the model and its necessary components onto the GPU for faster training, set the <code>Adam</code> optimizer, and configure the <code>CrossEntropyLoss</code> function to compute the differences between the translations output by the model and the reference translations from the dataset. </p>

<p style = "font-family: 'Helvetica Neue', Arial, sans-serif; text-align: left; font-size: 17.5px">Every loop necessary for iterating over the training batches, performing backpropagation, and computing the gradients is in this function. We will also use it to run the validation function and save the current state of the model.</p>

In [28]:
######################  TODO  ########################
######################  TODO  ########################

# - Define a `train_model` function to train a Transformer model
# - Steps:
#   1. Set up the device (GPU or CPU) for training
#   2. Create a directory to store model weights
#   3. Retrieve dataloaders and tokenizers for source and target languages using `get_ds`
#   4. Initialize the Transformer model using `get_model` and move it to the specified device
#   5. Set up TensorBoard for logging training metrics
#   6. Configure the Adam optimizer with learning rate and epsilon from `config`
#   7. If a pre-trained model exists:
#      - Load the model, optimizer state, and global step
#      - Set the starting epoch for resuming training
#   8. Define a cross-entropy loss function:
#      - Ignore padding tokens
#      - Apply label smoothing to prevent overfitting
#   9. Start training loop:
#      - Iterate over epochs from the initial epoch to `config['num_epochs']`
#      - For each batch in the training dataloader:
#         - Set model to training mode
#         - Move input data, masks, and labels to the device
#         - Pass data through the encoder, decoder, and projection layer
#         - Compute loss between model predictions and labels
#         - Log training loss to TensorBoard
#         - Perform backpropagation and update model parameters
#         - Clear gradients for the next batch
#         - Increment global step counter
#      - After each epoch, run validation using `run_validation`
#      - Save the current model state, optimizer state, and global step
#   10. Save model weights after each epoch

######################  TODO  ########################
######################  TODO  ########################
def train_model(config):
    # Define the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create the model folder if it doesn't exist
    Path(config['model_folder']).mkdir(parents=True, exist_ok=True)

    # Load the dataset
    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

    # Initialize the model
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    # Set up TensorBoard
    writer = SummaryWriter(config['experiment_name'])

    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

    # If a pre-trained model exists, load it
    initial_epoch = 0
    global_step = 0
    if config['preload'] is not None:
        model_filename = get_weights_file_path(config, config['preload'])
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']

    # Define the loss function
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    # Training loop
    for epoch in range(initial_epoch, config['num_epochs']):
        model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:
            encoder_input = batch['encoder_input'].to(device)  # (batch_size, seq_len)
            decoder_input = batch['decoder_input'].to(device)  # (batch_size, seq_len)
            encoder_mask = batch['encoder_mask'].to(device)  # (batch_size, 1, 1, seq_len)
            decoder_mask = batch['decoder_mask'].to(device)  # (batch_size, 1, seq_len, seq_len)

            # Run the tensors through the encoder, decoder, and projection layer
            encoder_output = model.encode(encoder_input, encoder_mask)  # (batch_size, seq_len, d_model)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)  # (batch_size, seq_len, d_model)
            proj_output = model.project(decoder_output)  # (batch_size, seq_len, vocab_size)

            # Get the label
            label = batch['label'].to(device)  # (batch_size, seq_len)

            # Compute the loss
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            # Log the loss
            writer.add_scalar('train_loss', loss.item(), global_step)
            writer.flush()

            # Backpropagate the loss
            loss.backward()

            # Update the weights
            optimizer.step()
            optimizer.zero_grad()

            global_step += 1

        # Run validation at the end of every epoch
        run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        # Save the model at the end of every epoch
        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

We can now train the model!

In [29]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    config = get_config()
    train_model(config)
    # pass

Using device: cuda


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274


...

--------------------------------------------------------------------------------
    SOURCE: "I am a fool!" cried Mr. Rochester suddenly. "I keep telling her I am not married, and do not explain to her why.
    TARGET: — Sono un pazzo! — esclamò a un tratto il signor Rochester. — Le dico che non sono ammogliato, e non le spiego il perché.
 PREDICTED: — Io , — disse il signor Rochester , — ma non ho detto che non ho detto che non ho detto che non posso essere mai .
--------------------------------------------------------------------------------
    SOURCE: It had lost half its tail, one of its ears, and a fairly appreciable proportion of its nose.
    TARGET: Aveva perduto la coda, un orecchio, e una parte del naso.
 PREDICTED: Era un ’ altra , e , e il suo , e .


Processing Epoch 02: 100%|██████████| 1213/1213 [14:00<00:00,  1.44it/s, loss=5.006]


--------------------------------------------------------------------------------
    SOURCE: I wondered why moralists call this world a dreary wilderness: for me it blossomed like a rose.
    TARGET: "Chiedevo a me stessa perché i filosofi chiamino il mondo un triste deserto; a me pareva pieno di fiori.
 PREDICTED: Io non mi a , e mi , mi .
--------------------------------------------------------------------------------
    SOURCE: Mr. Brocklehurst resumed. "This I learned from her benefactress; from the pious and charitable lady who adopted her in her orphan state, reared her as her own daughter, and whose kindness, whose generosity the unhappy girl repaid by an ingratitude so bad, so dreadful, that at last her excellent patroness was obliged to separate her from her own young ones, fearful lest her vicious example should contaminate their purity: she has sent her here to be healed, even as the Jews of old sent their diseased to the troubled pool of Bethesda; and, teachers, superinten

Processing Epoch 03: 100%|██████████| 1213/1213 [14:00<00:00,  1.44it/s, loss=5.122]


--------------------------------------------------------------------------------
    SOURCE: After making sure he had missed, he turned and saw that the trap and horses were no longer on the road but in the marsh.
    TARGET: Convinto d’aver fatto padella, Levin si voltò e vide che i cavalli col calesse non erano più sulla strada, ma nella palude.
 PREDICTED: Dopo aver visto il tempo , si era già già già già già già già già già già già più di lui , ma non era stato .
--------------------------------------------------------------------------------
    SOURCE: It's a banjo."
    TARGET: È un banjo.
 PREDICTED: È un po ’ di tè .


In [30]:
%ls

runs/  tokenizer_en.json  tokenizer_it.json  weights/


# Section 2: BERT and LoRA

Welcome to Section 2 of our Machine Learning assignment! I hope you've been enjoying the journey so far! 😊

 In this section, you will gain hands-on experience with [BERT](https://arxiv.org/abs/1810.04805) (Bidirectional Encoder Representations from Transformers) and [LoRA](https://arxiv.org/abs/2106.09685) (Low-Rank Adaptation) for text classification tasks. The section is divided into three main parts, each focusing on different aspects of NLP techniques.

## Assignment Structure

### Part 1: Data Preparation and Preprocessing
In this part, you will work with a text classification dataset. You will learn how to:
- Download and load the dataset
- Perform necessary preprocessing steps
- Implement data cleaning and transformation techniques
- Prepare the data in a format suitable for BERT training

### Part 2: Building a Small BERT Model
You will create and train a small BERT model from scratch using the Hugging Face [Transformers](https://huggingface.co/docs/transformers/en/index) library. This part will help you understand:
- The architecture of BERT
- How to configure and initialize a BERT model
- Training process and optimization
- Model evaluation and performance analysis

### Part 3: Fine-tuning with LoRA
In the final part, you will work with a pre-trained [TinyBERT](https://arxiv.org/abs/1909.10351) model and use LoRA for efficient fine-tuning. You will:
- Load a pre-trained TinyBERT model
- Implement LoRA adaptation and fine-tune the model on our classification task
- Compare the results with the previous approach

---

> **NOTE**:  
> Throughout this notebook, make an effort to include sufficient visualizations to enhance understanding:  
> - In the data processing section, display the results of your operations (e.g., show data samples or distributions after preprocessing).  
> - In the classification section, report various evaluation metrics such as accuracy, precision, recall, and F1-score to thoroughly assess your model's performance.  
> - Additionally, take a moment to compare the sizes of the models discussed in this notebook with today’s enormous models. This will help you appreciate the challenges and computational demands associated with training such massive models. 😵‍💫

---


## Part 1: Data Preparation and Preprocessing
We'll be working with the [Consumer Complaint](https://catalog.data.gov/dataset/consumer-complaint-database) dataset, which contains ***complaints*** submitted by consumers about financial products and services. Our goal is to build a classifier that can automatically identify the type of complaint based on the consumer's text description. For this task, we will work with a smaller subset of the dataset, available for download through this [link](https://drive.google.com/file/d/1SpIHksR-WzruEgUjp1SQKGG8bZPnJJoN/view?usp=sharing).

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import gdown
import string
import re
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

### 1.2 Loading the Data

In [2]:
file_id = "1SpIHksR-WzruEgUjp1SQKGG8bZPnJJoN"  # Replace this with your file's ID
output_file = "complaints_small.zip"  # Replace "data_file.ext" with the desired output filename and extension

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

Downloading...
From (original): https://drive.google.com/uc?id=1SpIHksR-WzruEgUjp1SQKGG8bZPnJJoN
From (redirected): https://drive.google.com/uc?id=1SpIHksR-WzruEgUjp1SQKGG8bZPnJJoN&confirm=t&uuid=8bda4fb9-bc74-4c6a-b499-d1b4e2a62d31
To: /content/complaints_small.zip
100%|██████████| 290M/290M [00:13<00:00, 20.8MB/s]


'complaints_small.zip'

In [3]:
!unzip complaints_small.zip

Archive:  complaints_small.zip
  inflating: complaints_small.csv    


In [4]:
%ls

complaints_small.csv  complaints_small.zip  sample_data/


In [5]:
######################  TODO  ########################
######################  TODO  ########################
# Load the dataset

######################  TODO  ########################
######################  TODO  ########################
df = pd.read_csv('complaints_small.csv')
df.head()

,Product,Consumer complaint narrative
0,"Credit reporting, credit repair services, or o...",My credit reports are inaccurate. These inaccu...
1,Student loan,Beginning in XX/XX/XXXX I had taken out studen...
2,Credit reporting or other personal consumer re...,I am disputing a charge-off on my account that...
3,"Credit reporting, credit repair services, or o...","I did not consent to, authorize, nor benefit f..."
4,Credit reporting or other personal consumer re...,I am a federally protected consumer and I am a...


### 1.3 Data Sampling and Class Distribution Analysis

Working with large datasets can be computationally intensive during development. Additionally, imbalanced class distribution can affect model performance. In this section, you'll sample the data and analyze class distributions to make informed decisions about your training dataset.

---

We'll work with a manageable portion of the data to develop and test our approach. While using the complete dataset would likely yield better results, a smaller sample allows us to prototype our solution more efficiently.


In [6]:
######################  TODO  ########################
######################  TODO  ########################

# - Sample a portion of the complete dataset
# - Display the first few rows of your sampled dataset
# - Print the shape of your original and sampled datasets

######################  TODO  ########################
######################  TODO  ########################
print("Original dataset shape:", df.shape)
df_sample = df.sample(n=10000, random_state=42)
print("Sampled dataset shape:", df_sample.shape)
df_sample.head(5)

Original dataset shape: (941128, 2)
Sampled dataset shape: (10000, 2)


,Product,Consumer complaint narrative
335123,Credit reporting or other personal consumer re...,"Upon reviewing my credit report, I have identi..."
601718,Mortgage,I was doing a rate check to refinance. The age...
847752,"Credit reporting, credit repair services, or o...",This is my 2nd request that I have been a vict...
765316,Credit reporting or other personal consumer re...,I'm sending this compliant to inform credit bu...
798300,"Credit reporting, credit repair services, or o...",Im submitting a complaint to you today to info...


---

Let's examine the distribution of ***complaints*** types in our dataset. You'll notice that some products have significantly more instances than others, and some categories are quite similar. For example:

- Multiple categories might refer to similar financial products
- Some categories might have very few examples
- Certain categories might be subcategories of others

You have two main approaches to handle this situation:

1. **Merging Similar Classes:** Identify categories that represent similar products/services and Combine them to create more robust, general categories

2. **Selecting Major Classes:** Only select the categories with sufficient representation



> You may choose any approach, but after this step, your data must include **at least five** distinct classes.



In [7]:
######################  TODO  ########################
######################  TODO  ########################

# - Display the number of complaints in each product category
# - Identify which classes are under-represented

# - Handle class imbalance by choosing and implementing one of these approaches:
#   1. Merge similar product categories (e.g., combining related categories)
#   2. Keep only the major classes with sufficient examples

######################  TODO  ########################
######################  TODO  ########################
# Analyze class distribution
product_counts = df_sample['Product'].value_counts()
print(product_counts)
print("--------------------------------------------")

# Handle class imbalance by selecting major classes
top_products = product_counts.head(5).index.tolist()
df_filtered = df_sample[df_sample['Product'].isin(top_products)]
print(df_filtered['Product'].value_counts())

Product
Credit reporting, credit repair services, or other personal consumer reports    3366
Credit reporting or other personal consumer reports                             2675
Debt collection                                                                 1300
Mortgage                                                                         520
Checking or savings account                                                      487
Credit card or prepaid card                                                      455
Credit card                                                                      241
Money transfer, virtual currency, or money service                               222
Student loan                                                                     192
Vehicle loan or lease                                                            164
Credit reporting                                                                 123
Payday loan, title loan, or personal loan                

---
### 1.4 Data Encoding and Text Preprocessing

Before training our model, we need to prepare both our target labels and text data. This involves converting categorical labels into numerical format and cleaning our text data to improve model performance.

In [8]:
######################  TODO  ########################
######################  TODO  ########################

# Label Encoding
# - Apply label encoding to convert product categories into numeric values

# Text Preprocessing
# Choose and implement preprocessing steps that you think will improve the quality of your text data.
# Here are some suggestions:

# - Remove special characters and punctuation
# - Remove very short complaints (e.g., less than 10 words)
# - Remove HTML tags if present

######################  TODO  ########################
######################  TODO  ########################
# Label Encoding
label_encoder = LabelEncoder()
df_filtered['label'] = label_encoder.fit_transform(df_filtered['Product'])
print(label_encoder.classes_)

# Text Preprocessing
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\b\w{1,2}\b', '', text)  # remove words with less than 3 letters
    return text.strip()

df_filtered['clean_complaint'] = df_filtered['Consumer complaint narrative'].apply(clean_text)

# Remove very short complaints
df_filtered = df_filtered[df_filtered['clean_complaint'].apply(lambda x: len(x.split()) >= 10)]
print(df_filtered.shape)

['Checking or savings account'
 'Credit reporting or other personal consumer reports'
 'Credit reporting, credit repair services, or other personal consumer reports'
 'Debt collection' 'Mortgage']


<ipython-input-8-4fee44703692>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['label'] = label_encoder.fit_transform(df_filtered['Product'])


(8184, 4)


<ipython-input-8-4fee44703692>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['clean_complaint'] = df_filtered['Consumer complaint narrative'].apply(clean_text)


## 1.5 Dataset Creation and Tokenization

For training our BERT model, we need to:
1. Create a custom Dataset class that will handle tokenization
2. Split the data into training and testing sets
3. Use BERT's tokenizer to convert text into a format suitable for the model

In [9]:
######################  TODO  ########################
######################  TODO  ########################

# class ComplaintDataset(Dataset):
#     """A custom Dataset class for handling consumer complaints text data with BERT tokenization.

#     Parameters:
#         texts (List[str]): List of complaint texts to be processed
#         labels (List[int]): List of encoded labels corresponding to each text
#         tokenizer (BertTokenizer): A BERT tokenizer instance for text processing
#         max_len (int, optional): Maximum length for padding/truncating texts. Defaults to 512

#     Returns:
#         dict: For each item, returns a dictionary containing:
#             - input_ids (torch.Tensor): Encoded token ids of the text
#             - attention_mask (torch.Tensor): Attention mask for the padded sequence
#             - labels (torch.Tensor): Encoded label as a tensor
#     """
#     def __init__(self, texts, labels, tokenizer, max_len=512):
#         ## TODO ##
#         pass

#     def __len__(self):
#         ## TODO ##
#         pass

#     def __getitem__(self, idx):
#         ## TODO ##
#         pass

######################  TODO  ########################
######################  TODO  ########################
class ComplaintDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [10]:
######################  TODO  ########################
######################  TODO  ########################

# # Split the data
# X_train, X_test, y_train, y_test = # TODO

# # Initialize tokenizer and create datasets
# tokenizer = # TODO
# train_dataset = # TODO
# test_dataset = # TODO

# # Create dataloaders
# train_loader = # TODO
# test_loader = # TODO

######################  TODO  ########################
######################  TODO  ########################
# Split the data
texts = df_filtered['clean_complaint'].tolist()
labels = df_filtered['label'].tolist()
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer and create datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ComplaintDataset(X_train, y_train, tokenizer)
test_dataset = ComplaintDataset(X_test, y_test, tokenizer)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Part 2: Training a Small-Size BERT Model

In this part, we will explore how to build and train a small-sized BERT model for our classification task. Instead of using the full-sized BERT model, which is computationally expensive, we will create a smaller version using the Transformers library.

In [15]:
######################  TODO  ########################
######################  TODO  ########################

# 1. Define your BERT model for sequence classification
#    Ensure that you set up the configuration properly (e.g., specify the number of output labels).
# 2. Print the total number of trainable parameters in the model to understand its size.

######################  TODO  ########################
######################  TODO  ########################

# Define BERT model for sequence classification
# config = BertConfig(
#     hidden_size=128,
#     num_hidden_layers=2,
#     num_attention_heads=4,
#     intermediate_size=512,
#     num_labels=len(label_encoder.classes_)
# )
# model = BertForSequenceClassification(config)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
print("Number of trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 109486085


---

Now that you have defined your model, it's time to train it!☠️

Training a model of this size can take some time, depending on the available resources. To manage this, you can train your model for just **2–3 epochs** to demonstrate progress. Here are some hints:
- **Training Metrics:** Ensure you print enough metrics, such as loss and accuracy, to track the training progress.
- **Interactive Monitoring:** Use the `tqdm` library to display the progress of your training loop in real-time.

In [16]:
######################  TODO  ########################
######################  TODO  ########################

# optimizer = # TODO
# num_epochs = # TODO

# # Training loop
# for epoch in range(num_epochs):

#     model.train()

#     for batch in tqdm(train_loader):
#         optimizer.zero_grad()

#         input_ids = # TODO
#         attention_mask = # TODO
#         labels = # TODO

#         outputs = model(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             labels=labels
#         )

#         # TODO: Perform backpropagation and update the optimizer. Hint: Use outputs.loss to access the model's loss.

#         # TODO: Monitor the training process by reporting metrics such as loss and accuracy.


# # TODO : Evaluate the model on test dataset

######################  TODO  ########################
######################  TODO  ########################
# Set up optimizer and training loop
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Train Loss: {avg_loss:.4f}, Train Accuracy: {100 * correct / total:.2f}%")

    # Evaluation
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        # for batch in test_loader:
        for batch in tqdm(test_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = total_val_loss / len(test_loader)
    print(f"Epoch {epoch + 1}, Average Val Loss: {avg_val_loss:.4f}, Val Accuracy: {100 * correct / total:.2f}%")

# TODO : Evaluate the model on test dataset
print("Evaluating the model on test dataset")
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, predicted = torch.max(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

100%|██████████| 205/205 [10:40<00:00,  3.12s/it]


Epoch 1, Average Train Loss: 1.0349, Train Accuracy: 54.10%


100%|██████████| 52/52 [00:58<00:00,  1.12s/it]


Epoch 1, Average Val Loss: 0.7287, Val Accuracy: 69.82%


100%|██████████| 205/205 [10:40<00:00,  3.12s/it]


...

Epoch 3, Average Train Loss: 0.5152, Train Accuracy: 79.38%


100%|██████████| 52/52 [00:58<00:00,  1.13s/it]


Epoch 3, Average Val Loss: 0.6600, Val Accuracy: 74.22%
Evaluating the model on test dataset


100%|██████████| 52/52 [00:58<00:00,  1.13s/it]

Test Accuracy: 0.7422


## Part 3: Fine-Tuning TinyBERT with LoRA

As you have experienced, training even a small-sized BERT model can be computationally intensive and time-consuming. To address these challenges, we explore **Parameter-Efficient Fine-Tuning (PEFT)** methods, which allow us to utilize the power of large pretrained models without requiring extensive resources.

---

### **Parameter-Efficient Fine-Tuning (PEFT)**

PEFT methods focus on fine-tuning only a small portion of the model’s parameters while keeping most of the pretrained weights frozen. This drastically reduces the computational and storage requirements while leveraging the rich knowledge embedded in pretrained models.

One popular PEFT method is LoRA (Low-Rank Adaptation).

- **What is LoRA?**

LoRA introduces a mechanism to fine-tune large language models by injecting small low-rank matrices into the model's architecture. Instead of updating all parameters during training, LoRA trains these small matrices while keeping the majority of the original parameters frozen.  This is achieved as follows:

1. **Frozen Weights**: The pretrained weights of the model, represented as a weight matrix $ W \in \mathbb{R}^{d \times k} $, remain **frozen** during fine-tuning.

2. **Low-Rank Decomposition**:
   Instead of directly updating $ W $, LoRA introduces two trainable matrices, $ A \in \mathbb{R}^{d \times r} $ and $ B \in \mathbb{R}^{r \times k} $, where $ r \ll \min(d, k) $.  
   These matrices approximate the update to $ W $ as:
   $$
   \Delta W = A \cdot B
   $$

   Here, $ r $, the rank of the decomposition, is a key hyperparameter that determines the trade-off between computational cost and model capacity.

3. **Adaptation**:
   During training, instead of updating $ W $, the adapted weight is:
   $$
   W' = W + \Delta W = W + A \cdot B
   $$
   Only the low-rank matrices $ A $ and $ B $ are optimized, while $ W $ remains fixed.

4. **Efficiency**:
   Since $ r $ is much smaller than $ d $ and $ k $, the number of trainable parameters in $ A $ and $ B $ is significantly less than in $ W $. This makes the approach highly efficient both in terms of computation and memory.

---

###  **Fine-Tuning TinyBERT**

For this part, we will fine-tune **TinyBERT**, a distilled version of BERT, using the LoRA method.

- **What is TinyBERT?**

TinyBERT is a lightweight version of the original BERT model created through knowledge distillation. It significantly reduces the model size and inference latency while preserving much of the original BERT’s effectiveness. Here are some key characteristics of TinyBERT:
- It is designed to be more resource-efficient for tasks such as classification, question answering, and more.
- TinyBERT retains a compact structure with fewer layers and parameters, making it ideal for fine-tuning with limited computational resources.


> Similar to the previous section, training this model might take some time. Given the resource limitations, you can train the model for just **2-3 epochs** to demonstrate the process.


In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

In [27]:
######################  TODO  ########################
######################  TODO  ########################

# # Load the pre-trained TinyBERT
# model_name = "prajjwal1/bert-tiny"
# base_model = # TODO
# tokenizer = # TODO

# # Define LoRA Configuration
# lora_config = # TODO

######################  TODO  ########################
######################  TODO  ########################
# Load the pre-trained TinyBERT
model_name = "prajjwal1/bert-tiny"
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define LoRA Configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,  # Rank of the low-rank matrices
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    # target_modules=["query", "key", "value"], # Modules to apply LoRA
    target_modules=["query", "value"],
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
######################  TODO  ########################
######################  TODO  ########################

# # Apply LoRA to model
# lora_model = # TODO

# # TODO: Show the number of trainable parameters

# # Training configuration
# optimizer = # TODO
# criterion = # TODO

######################  TODO  ########################
######################  TODO  ########################
# Apply LoRA to model
lora_model = get_peft_model(base_model, lora_config)

# Show the number of trainable parameters
trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

# Training configuration
optimizer = AdamW(lora_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

Trainable parameters: 8837


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
######################  TODO  ########################
######################  TODO  ########################

# optimizer = # TODO
# num_epochs = # TODO

# # Training loop
# for epoch in range(num_epochs):

#     lora_model.train()

#     for batch in tqdm(train_loader):
#         optimizer.zero_grad()

#         input_ids = # TODO
#         attention_mask = # TODO
#         labels = # TODO

#         outputs = lora_model(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             labels=labels
#         )

#         # TODO: Perform backpropagation and update the optimizer. Hint: Use outputs.loss to access the model's loss.

#         # TODO: Monitor the training process by reporting metrics such as loss and accuracy.


# # TODO : Evaluate the model on test dataset

######################  TODO  ########################
######################  TODO  ########################

optimizer = AdamW(lora_model.parameters(), lr=1e-3)
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lora_model.to(device)
# Training loop
for epoch in range(num_epochs):
    lora_model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = lora_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Train Loss: {avg_loss:.4f}, Train Accuracy: {100 * correct / total:.2f}%")

    # Evaluation
    lora_model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        # for batch in test_loader:
        for batch in tqdm(test_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = lora_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels  # Add labels here
            )

            loss = outputs.loss
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = total_val_loss / len(test_loader)
    print(f"Epoch {epoch + 1}, Average Val Loss: {avg_val_loss:.4f}, Val Accuracy: {100 * correct / total:.2f}%")

print("Evaluating the model on test dataset")
# TODO : Evaluate the model on test dataset
lora_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = lora_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, predicted = torch.max(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 205/205 [00:48<00:00,  4.19it/s]


Epoch 1, Average Train Loss: 1.1119, Train Accuracy: 48.04%


100%|██████████| 52/52 [00:07<00:00,  7.21it/s]


Epoch 1, Average Val Loss: 0.9871, Val Accuracy: 52.72%


100%|██████████| 205/205 [00:43<00:00,  4.76it/s]


...

Epoch 5, Average Train Loss: 0.8142, Train Accuracy: 63.57%


100%|██████████| 52/52 [00:07<00:00,  6.71it/s]


Epoch 5, Average Val Loss: 0.8061, Val Accuracy: 65.79%
Evaluating the model on test dataset


100%|██████████| 52/52 [00:09<00:00,  5.41it/s]

Test Accuracy: 0.6579
